In [1]:
import pandas as pd
import os

└── training_data
    ├── degradation_data
    │   └──item_(n).csv            // n = 50 files
    ├── failure_data.csv
    ├── pseudo_testing_data
    │   └── item_(n).csv            // n = 50 files
    └── pseudo_testing_data_with_truth
        ├── Solution.csv
        └── item_(n).csv            // n = 50 files

Dans un dossier training data :  

└── training_data
    ├── degradation_data
    │   └──item_(n).csv            // n = 50 files
    ├── failure_data.csv

Les fichiers sont structurés comme suit : 

* failure_data.csv :  résumé des temps jusqu'à la panne pour les 50 échantillons.

    - Type int    --> item_id
    - Type int    --> Time to failure (months)
    - Type string --> Failure mode

    // Indique le mode de défaillance pour chaque échantillon :
    // 'Infant Mortality', 'Fatigue Crack', ou 'Control Board Failure'

* Degradation data (a folder with x50 .csv files):  mesures de la longueur de fissure pour les 50 échantillons.  

    - Type int   --> time (months)  
    - Type float --> crack length (arbitary unit)  
    - Type int   --> rul (months)  

    // Chaque fichier CSV dans ce dossier correspond à un échantillon spécifique.
    // Le nom du fichier `item_X` correspond à l'identifiant de l'échantillon (`item_id`) dans le fichier `failure_data.csv`.


Est-ce que tu peux me proposer du code en Python pour concaténer toutes les données en un dataframe, de façon logique ?

In [7]:

def training_data_expàloration():
    
    failure_data_path = '../../data/input/training_data/failure_data.csv'
    degradation_data_path = '../../data/input/training_data/degradation_data'
    
    failure_data = pd.read_csv(failure_data_path)
    
    degradation_dfs = []    # Liste pour stocker les DataFrames des données de dégradation

    for filename in os.listdir(degradation_data_path):  # Lire chaque fichier de dégradation et les ajouter à la liste
        if filename.endswith('.csv'):
            item_id = int(filename.split('_')[1].split('.')[0])
            file_path = os.path.join(degradation_data_path, filename)
            df = pd.read_csv(file_path)
            df['item_id'] = item_id
            degradation_dfs.append(df)

    degradation_data = pd.concat(degradation_dfs)       # Concaténer tous les DataFrames de dégradation en un seul DataFrame

    combined_data = pd.merge(degradation_data, failure_data, on='item_id', how='left')      # Fusionner les données de dégradation avec les données de failure_data sur 'item_id'

    return failure_data, degradation_data, combined_data

failure_data, degradation_data, combined_data = training_data_expàloration()

In [10]:
failure_data.head(10)


,item_id,Time to failure (months),Failure mode
0,0,2,Infant mortality
1,1,1,Infant mortality
2,2,1,Infant mortality
3,3,2,Infant mortality
4,4,2,Infant mortality
5,5,38,Fatigue crack
6,6,36,Fatigue crack
7,7,45,Fatigue crack
8,8,30,Fatigue crack
9,9,42,Fatigue crack


In [11]:
degradation_data.head(10)

,time (months),crack length (arbitary unit),rul (months),item_id
0,0,0.003686,39,16
1,1,-0.014691,38,16
2,2,-0.048912,37,16
3,3,-0.000898,36,16
4,4,-0.007360,35,16
5,5,0.016732,34,16
6,6,-0.009443,33,16
7,7,0.026804,32,16
8,8,-0.015147,31,16
9,9,0.009385,30,16


In [12]:
combined_data.head(10)

,time (months),crack length (arbitary unit),rul (months),item_id,Time to failure (months),Failure mode
0,0,0.003686,39,16,39.0,Fatigue crack
1,1,-0.014691,38,16,39.0,Fatigue crack
2,2,-0.048912,37,16,39.0,Fatigue crack
3,3,-0.000898,36,16,39.0,Fatigue crack
4,4,-0.007360,35,16,39.0,Fatigue crack
5,5,0.016732,34,16,39.0,Fatigue crack
6,6,-0.009443,33,16,39.0,Fatigue crack
7,7,0.026804,32,16,39.0,Fatigue crack
8,8,-0.015147,31,16,39.0,Fatigue crack
9,9,0.009385,30,16,39.0,Fatigue crack


In [14]:
import matplotlib.pyplot as plt

def plot_crack_length_by_item(df):
    """
    Trace la courbe de l'évolution de la longueur de fissure en fonction du temps pour chaque item_id unique.
    
    Parameters:
    df (pd.DataFrame): DataFrame contenant les colonnes 'time', 'crack length', et 'item_id'.
    """
    # Créer une figure avec un sous-ensemble d'axes
    plt.figure(figsize=(12, 8))
    
    # Obtenir les item_ids uniques
    item_ids = df['item_id'].unique()
    
    for item_id in item_ids:
        # Filtrer les données pour l'item_id courant
        item_df = df[df['item_id'] == item_id]
        
        # Tracer la courbe pour cet item_id
        plt.plot(item_df['time (months)'], item_df['crack length (arbitary unit)'], label=f'Item {item_id}')
    
    # Ajouter des labels et un titre
    plt.xlabel('Time (months)')
    plt.ylabel('Crack Length (arbitrary unit)')
    plt.title('Crack Length Evolution by Item')
    plt.legend()
    plt.grid(True)
    
    # Afficher le graphique
    plt.show()

# Exemple d'utilisation
plot_crack_length_by_item(degradation_data)


KeyError: 'crack length'

<Figure size 1200x800 with 0 Axes>